In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import datetime
import re
import win32com.client  #pip install pywin32
from generate_emails import *
from datetime import datetime, timedelta
import random
import json


API key found in environment variables.


## Generate random dates for the synthetic emails

In [10]:
# Set the range of the past week
start_date = datetime.now() - timedelta(days=7)
end_date = datetime.now()

# Generate random datetime objects within the range
random_dates = []

num_of_dates = 20

for _ in range(num_of_dates):  # Generate 10 random datetime objects
    random_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    random_dates.append(random_date)

# Print the generated random datetime objects
for date in random_dates:
    print(date)

2024-05-01 17:11:26.738746
2024-04-28 10:34:09.738746
2024-05-01 14:27:02.738746
2024-04-29 19:21:26.738746
2024-04-28 12:58:33.738746
2024-04-28 08:40:49.738746
2024-04-30 04:03:58.738746
2024-05-01 21:28:08.738746
2024-04-30 18:34:22.738746
2024-04-28 15:10:46.738746
2024-04-28 04:56:44.738746
2024-05-02 19:26:11.738746
2024-05-02 23:19:27.738746
2024-05-03 05:50:46.738746
2024-04-30 17:17:55.738746
2024-04-29 14:12:14.738746
2024-05-04 17:34:50.738746
2024-05-02 10:16:20.738746
2024-05-02 19:46:11.738746
2024-05-01 13:50:51.738746


In [4]:
import json

# Conversation between Alice and Bob
conversation = [
    {
        "sender": "Alice",
        "message": "Hey Bob! I'm super excited about the upcoming hackathon, HackUPC! Are you still up for going to Barcelona with me?"
    },
    {
        "sender": "Bob",
        "message": "Hey Alice! Absolutely, I've been looking forward to it! When are we planning to head to Barcelona?"
    },
    {
        "sender": "Alice",
        "message": "Great! I was thinking we could fly in a day before the hackathon starts. How does that sound to you?"
    },
    {
        "sender": "Bob",
        "message": "That sounds perfect to me. We can use that extra day to explore the city and get a feel for the surroundings. Do you have any preferences for accommodation?"
    },
    {
        "sender": "Alice",
        "message": "I was thinking of booking an Airbnb near the venue. It would be convenient for us and save us some travel time. What do you think?"
    },
    {
        "sender": "Bob",
        "message": "That's a good idea. It'll be nice to have our own space during the hackathon. Let's go ahead and book it. Have you looked into flights yet?"
    },
    {
        "sender": "Alice",
        "message": "Not yet, but I'll check for flights today and let you know. We should book them as soon as possible to get good deals. Also, do you have any dietary preferences I should keep in mind?"
    },
    {
        "sender": "Bob",
        "message": "I'm pretty flexible with food, but thanks for asking. Let's make sure to try some authentic Spanish cuisine while we're there! I'm getting excited just thinking about it."
    },
    {
        "sender": "Alice",
        "message": "Me too! It's going to be an amazing experience. I'll send you the flight options as soon as I find them. Anything else we need to plan before the trip?"
    },
    {
        "sender": "Bob",
        "message": "I think we've covered everything for now. If anything else comes up, we can discuss it later. Thanks for taking the lead on planning, Alice. You're the best!"
    },
    {
        "sender": "Alice",
        "message": "No problem, Bob! We make a great team. Get ready for an awesome hackathon and an unforgettable trip to Barcelona!"
    }
]

# Output the conversation in JSON format
with open("./Input/threads.json", "w") as file:
    json.dump(conversation, file, indent=4)

print("Conversation saved to travel_plans_conversation.json")

Conversation saved to travel_plans_conversation.json


In [11]:
email_threads_topics = ["Developing Markets", "eFx", "US Equity Markets", "Fixed Income","Commodities"]
authors = ["JP Morgan", "Goldman Sachs", "Morgan Stanley", "Bank of America"]
emails = ["jpmorgan@upc.edu", "goldmansachs@upc.edu", "morganstanley@upc.edu", "boa@upc.edu"]

recipients = ["hackupc-test@outlook.com"]

email_count = 0
num_emails = 2



In [12]:

message_list = []

for j in range(len(authors)):
    for topic in email_threads_topics:

        message = {}
        message['email_date'] = random_dates[email_count%len(random_dates)]
        message['sender'] = emails[j]
        message['recipient'] = recipients[0]
        message['subject'] = f"{authors[j]}: {topic} Update and Trading Ideas"
        message['email_id'] = email_count


        message['text'] = generate_email_body(topic, authors[j], previous_emails=None)

        #save_mbox(thread, f"./emails/{topic.replace(' ', '_').lower()}.txt")

        email_count += 1
        message_list.append(message)


input_dir = Path.cwd() / "Input"
with open(input_dir / "emails.json", "w") as f:
    json.dump(message_list, f, indent=4, default=str)

## Populate the emails in the inbox

In [13]:
# Get the hackupc-test@outlook.com account
account = None

outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

for acc in namespace.Accounts:
    if acc.DisplayName == 'hackupc-test@outlook.com':
        account = acc
        break

if account is None:
    print("Hackupc account not found.")
else:
    # Iterate over the message_list and send emails
    for message in message_list:
        mail = outlook.CreateItem(0)  # Create a new email
        mail.Subject = message['subject']
        mail.Body = message['text']
        mail.To = message['recipient']
        mail.SendUsingAccount = account  # Set the account to use for sending
        mail.Send()


## Parse the emails from the Outlook Inbox

In [18]:


# Create output folder
output_dir = Path.cwd() / "Output"
output_dir.mkdir(parents=True, exist_ok=True)

# Connect to outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Connect to folder
#inbox = outlook.Folders('arvind.menon@epfl.ch').Folders('Inbox')
inbox = outlook.Folders('hackupc-test@outlook.com').Folders('Inbox')
# inbox = outlook.GetDefaultFolder(6)

# https://docs.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
# DeletedItems=3, Outbox=4, SentMail=5, Inbox=6, Drafts=16, FolderJunk=23

# Get messages
messages = inbox.Items

list_of_messages = []

for message in messages:
    subject = message.Subject
    body = message.body
    # attachments = message.Attachments
    recipients = message.Recipients
    sender = message.Sender
    date = message.SentOn
    entry_id = message.EntryID
    # Create separate folder for each message, exclude special characters and timestampe

    msg = dict()
    msg["subject"] = getattr(message, "Subject", "<UNKNOWN>")
    msg["recipient"] = [r.Name for r in message.Recipients]
    
    msg["email_date"] = getattr(message, "SentOn", "<UNKNOWN>")
    msg["email_id"] = getattr(message, "EntryID", "<UNKNOWN>")
    msg["sender"] = getattr(message, "Sender", "<UNKNOWN>")
    #msg["Size"] = getattr(message, "Size", "<UNKNOWN>")
    msg["text"] = getattr(message, "Body", "<UNKNOWN>")

    list_of_messages.append(msg)

import json
with open(output_dir / "emails_wthread.json", "w") as f:
    json.dump(list_of_messages, f, indent=4, default=str)

## Print the extracted elements from the inbox

In [17]:
def extract(count):
    """Get emails from outlook."""
    items = []
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders('hackupc-test@outlook.com').Folders('Inbox')
    messages = inbox.Items
    message = messages.GetFirst()
    i = 0
    while message:
        try:
            msg = dict()
            msg["Subject"] = getattr(message, "Subject", "<UNKNOWN>")
            msg["Recipients"] = [r.Name for r in message.Recipients]
            
            msg["SentOn"] = getattr(message, "SentOn", "<UNKNOWN>")
            msg["EntryID"] = getattr(message, "EntryID", "<UNKNOWN>")
            msg["Sender"] = getattr(message, "Sender", "<UNKNOWN>")
            msg["Size"] = getattr(message, "Size", "<UNKNOWN>")
            msg["Body"] = getattr(message, "Body", "<UNKNOWN>")
            items.append(msg)
        except Exception as ex:
            print("Error processing mail", ex)
        i += 1
        if i < count:
            message = messages.GetNext()
        else:
            return items

    return items

def show_message(items):
    """Show the messages."""
    items.sort(key=lambda tup: tup["SentOn"])
    for i in items:
        print(i["SentOn"], i["Subject"], i["Recipients"])


def main():
    """Fetch and display top message."""
    items = extract(5)
    show_message(items)


if __name__ == "__main__":
    main()

API key found in environment variables.
2024-05-04 23:28:01+00:00 JP Morgan: Developing Markets Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 23:28:04+00:00 Goldman Sachs: US Equity Markets Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 23:28:06+00:00 Morgan Stanley: US Equity Markets Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 23:28:07+00:00 Morgan Stanley: Commodities Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 23:28:08+00:00 Bank of America: US Equity Markets Update and Trading Ideas ['hackupc-test@outlook.com']


## Generate a thread using the conversation JSON

In [14]:
# load a json

with open("./Input/threads.json", "r") as file:
    conversation = json.load(file)

print(conversation)

[{'sender': 'Alice', 'message': "Hey Bob! I'm super excited about the upcoming hackathon, HackUPC! Are you still up for going to Barcelona with me?"}, {'sender': 'Bob', 'message': "Hey Alice! Absolutely, I've been looking forward to it! When are we planning to head to Barcelona?"}, {'sender': 'Alice', 'message': 'Great! I was thinking we could fly in a day before the hackathon starts. How does that sound to you?'}, {'sender': 'Bob', 'message': 'That sounds perfect to me. We can use that extra day to explore the city and get a feel for the surroundings. Do you have any preferences for accommodation?'}, {'sender': 'Alice', 'message': 'I was thinking of booking an Airbnb near the venue. It would be convenient for us and save us some travel time. What do you think?'}, {'sender': 'Bob', 'message': "That's a good idea. It'll be nice to have our own space during the hackathon. Let's go ahead and book it. Have you looked into flights yet?"}, {'sender': 'Alice', 'message': "Not yet, but I'll ch

In [16]:
import win32com.client as win32
import json

# Get the hackupc-test@outlook.com account
account = None

outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

for acc in namespace.Accounts:
    if acc.DisplayName == 'hackupc-test@outlook.com':
        account = acc
        break

recipient_email = 'hackupc-test@outlook.com'
if account is None:
    print("Hackupc account not found.")
else:
    # Iterate over the message_list and send emails
    for message in conversation:
        mail = outlook.CreateItem(0)  # Create a new email
        mail.Subject = "Conversation about HackUPC Travel Plans"
        mail.Body = message["message"]
        mail.To = recipient_email
        mail.SendUsingAccount = account  # Set the account to use for sending
        mail.Send()


In [11]:
for i in outlook.Session.Accounts:

    print(i)

arvind.menon@epfl.ch
hackupc-test@outlook.com
